<a href="https://colab.research.google.com/github/tedyadika/Azureml/blob/main/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install azureml.core
!pip install numpy == 1.20
!pip install pycaret
!pip install azureml-dataset-runtime --upgrade
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Invalid requirement: '=='
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import Azure ML SDK modules

In [62]:
from azureml.core import Workspace, Datastore,Dataset,Experiment,Run

### Import python modules

In [63]:
%matplotlib inline
import datetime
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date,timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [87]:
data_read ='country_sdk'
filter_column ='ORDTYP_map'
filter_value = "ECOMM"
col_lag_list = ['TOTAL_ORDNUM_LS','DW_SUM_ORDQTY_AD','DW_QTY_REC_LS','SUM_QTY_PIECE_PICKS','TOTAL_ORDNUM_AD']
split_column= 'COUNTRY_NAME'
target_column = 'SUM_QTY_PIECE_PICKS'
cols_drop = ['SUM_QTY_PIECE_PICKS','NBR_ORDERS','SUM_QTY_TOTAL_PICKS','DW_SUM_ORDQTY_AD','DW_QTY_REC_LS','TOTAL_ORDNUM_LS','TOTAL_ORDNUM_AD']

### Set up and data import 

In [88]:
ws = Workspace.get(name= 'dscglobalceedamlt',
                    subscription_id= '8b20e38c-72fb-4030-a702-128e713479ca',
                    resource_group= 'DSCGlobalCEEDTANLTst')

az_store= Datastore.get(ws,'dscglobalceedamlsat_datastore')
az_dataset= Dataset.get_by_name(ws,data_read)
az_default_store = ws.get_default_datastore()

### set up experiment

In [89]:
experiment_run = Run.get_context()
dataset1 =az_dataset.to_pandas_dataframe()

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


In [90]:
df = dataset1.copy()
if filter_column and filter_value !=None:
    df = df.query(f'{filter_column}  == "{filter_value}"')
elif filter_column and filter_value == None:
    output_log = 'NO_FILTER APPLIED'
    experiment_run.log(output_log)

In [91]:
nulldf = df.isnull().sum()
for col in df.columns:
    experiment_run.log(col, nulldf[col])

Attempted to log scalar metric DATE:
0
Attempted to log scalar metric ORDTYP_map:
0
Attempted to log scalar metric COUNTRY_NAME:
0
Attempted to log scalar metric NBR_ORDERS:
0
Attempted to log scalar metric DW_SUM_ORDQTY_AD:
5474
Attempted to log scalar metric DW_QTY_REC_LS:
1954
Attempted to log scalar metric TOTAL_ORDNUM_LS:
1715
Attempted to log scalar metric TOTAL_ORDNUM_AD:
5474
Attempted to log scalar metric SUM_QTY_TOTAL_PICKS:
0
Attempted to log scalar metric SUM_QTY_PIECE_PICKS:
0


### spliting our data by country 

In [92]:
# Initialize an empty list
df_list = []
# Split the dataframe into separate datasets based on the values in the 'country' column
for name, group in df.groupby(split_column):
  df_list.append(group)

### Adding importang parameters to our data

In [93]:
def add_values(data):
    #data = df.copy()
    data['DATE'] = pd.to_datetime(data['DATE'],format ='%Y-%m-%d', errors='coerce')
    data.sort_values('DATE', inplace=True)
    data['Month'] = [i.month for i in data['DATE']]
    data['Year'] = [i.year for i in data['DATE']]
    data['WEEK_YEAR'] = data['DATE'].dt.isocalendar().week 
    data['day_of_week'] = data['DATE'].dt.dayofweek
    data['DAY_YEAR'] = data['DATE'].dt.dayofyear
    data['DATE_N'] = data['DATE'].apply(lambda x:x.toordinal())
    data["WEEK_YEAR"] = data["WEEK_YEAR"].astype('int64').astype('int64')
    return(data)



In [100]:
new_col = ['logarithm_base2']
col_lag_list = col_lag_list + new_col
col_lag_list

['TOTAL_ORDNUM_LS',
 'DW_SUM_ORDQTY_AD',
 'DW_QTY_REC_LS',
 'SUM_QTY_PIECE_PICKS',
 'TOTAL_ORDNUM_AD',
 'logarithm_base2']

### Adding Lags

In [101]:
lag = [7,14,21,28,35]
def add_lags(df_list,column_list,lag_list):
  for i in range(len(df_list)):
      df = df_list[i]#for df in df_list:
      df = df.dropna(subset=['DATE'])
      add_values(df)
      for col in col_lag_list:
        for l in lag:
          new_col_name = col + '_lag_' + str(l)
          new_col_name2 = col + '_mean_' + str(l)
          df_list[i][new_col_name] = df_list[i][col].shift(l)
          df_list[i][new_col_name2] = df_list[i][col].shift(l).rolling(7).mean()
          df_list[i] = df_list[i].fillna(df.mean())


add_lags(df_list,col_lag_list,lag)


<ipython-input-101-f470a8e88c3f>:13: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-101-f470a8e88c3f>:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-101-f470a8e88c3f>:13: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-101-f470a8e88c3f>:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df

In [102]:
for data in df_list:
  data.sort_values('DATE', inplace=True)
  data['logarithm_base2'] = np.log10(data[target_column])
  data['MA14'] = data[target_column].shift(14).rolling(7).mean()
  data['MA7'] = data[target_column].shift(7).rolling(14).mean()


In [103]:
pd.set_option('display.max_columns', None)
df_list[0]

,DATE,ORDTYP_map,COUNTRY_NAME,NBR_ORDERS,DW_SUM_ORDQTY_AD,DW_QTY_REC_LS,TOTAL_ORDNUM_LS,TOTAL_ORDNUM_AD,SUM_QTY_TOTAL_PICKS,SUM_QTY_PIECE_PICKS,TOTAL_ORDNUM_LS_lag_7,TOTAL_ORDNUM_LS_mean_7,TOTAL_ORDNUM_LS_lag_14,TOTAL_ORDNUM_LS_mean_14,TOTAL_ORDNUM_LS_lag_21,TOTAL_ORDNUM_LS_mean_21,TOTAL_ORDNUM_LS_lag_28,TOTAL_ORDNUM_LS_mean_28,TOTAL_ORDNUM_LS_lag_35,TOTAL_ORDNUM_LS_mean_35,DW_SUM_ORDQTY_AD_lag_7,DW_SUM_ORDQTY_AD_mean_7,DW_SUM_ORDQTY_AD_lag_14,DW_SUM_ORDQTY_AD_mean_14,DW_SUM_ORDQTY_AD_lag_21,DW_SUM_ORDQTY_AD_mean_21,DW_SUM_ORDQTY_AD_lag_28,DW_SUM_ORDQTY_AD_mean_28,DW_SUM_ORDQTY_AD_lag_35,DW_SUM_ORDQTY_AD_mean_35,DW_QTY_REC_LS_lag_7,DW_QTY_REC_LS_mean_7,DW_QTY_REC_LS_lag_14,DW_QTY_REC_LS_mean_14,DW_QTY_REC_LS_lag_21,DW_QTY_REC_LS_mean_21,DW_QTY_REC_LS_lag_28,DW_QTY_REC_LS_mean_28,DW_QTY_REC_LS_lag_35,DW_QTY_REC_LS_mean_35,SUM_QTY_PIECE_PICKS_lag_7,SUM_QTY_PIECE_PICKS_mean_7,SUM_QTY_PIECE_PICKS_lag_14,SUM_QTY_PIECE_PICKS_mean_14,SUM_QTY_PIECE_PICKS_lag_21,SUM_QTY_PIECE_PICKS_mean_21,SUM_QTY_PIECE_PICKS_lag_28,SUM_QTY_PIECE_PICKS_mean_28,SUM_QTY_PIECE_PICKS_lag_35,SUM_QTY_PIECE_PICKS_mean_35,TOTAL_ORDNUM_AD_lag_7,TOTAL_ORDNUM_AD_mean_7,TOTAL_ORDNUM_AD_lag_14,TOTAL_ORDNUM_AD_mean_14,TOTAL_ORDNUM_AD_lag_21,TOTAL_ORDNUM_AD_mean_21,TOTAL_ORDNUM_AD_lag_28,TOTAL_ORDNUM_AD_mean_28,TOTAL_ORDNUM_AD_lag_35,TOTAL_ORDNUM_AD_mean_35,logarithm_base2,MA14,MA7,logarithm_base2_lag_7,logarithm_base2_mean_7,logarithm_base2_lag_14,logarithm_base2_mean_14,logarithm_base2_lag_21,logarithm_base2_mean_21,logarithm_base2_lag_28,logarithm_base2_mean_28,logarithm_base2_lag_35,logarithm_base2_mean_35
1974,2019-08-28,ECOMM,AUT,6,23570.655319,21303.000000,11860.000000,1872.259574,6,6,7358.600642,7694.254317,7361.804970,7358.069654,7351.070635,7357.443386,7358.539832,7361.253986,7351.790642,7361.253158,23589.541125,23587.896243,23570.809137,23566.185747,23559.078592,23564.091116,23567.974514,23567.808841,23567.692157,23566.422486,14226.730489,14239.927737,14232.587368,14233.739231,14238.266471,14248.617194,14240.853276,14259.348142,14257.956495,14271.286100,622.585028,623.581825,623.270867,624.454417,624.816461,626.164126,626.118377,626.105657,625.686095,626.871608,1873.780534,1874.402735,1876.599001,1876.037480,1876.887803,1875.801928,1875.876145,1876.735300,1877.370379,1877.116812,0.778151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4990,2019-09-04,ECOMM,AUT,6,23570.655319,24969.000000,12996.000000,1872.259574,12,12,7358.600642,7694.254317,7361.804970,7358.069654,7351.070635,7357.443386,7358.539832,7361.253986,7351.790642,7361.253158,23589.541125,23587.896243,23570.809137,23566.185747,23559.078592,23564.091116,23567.974514,23567.808841,23567.692157,23566.422486,14226.730489,14239.927737,14232.587368,14233.739231,14238.266471,14248.617194,14240.853276,14259.348142,14257.956495,14271.286100,622.585028,623.581825,623.270867,624.454417,624.816461,626.164126,626.118377,626.105657,625.686095,626.871608,1873.780534,1874.402735,1876.599001,1876.037480,1876.887803,1875.801928,1875.876145,1876.735300,1877.370379,1877.116812,1.079181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13790,2019-09-05,ECOMM,AUT,15,23570.655319,26193.000000,3629.000000,1872.259574,43,43,7358.600642,7694.254317,7361.804970,7358.069654,7351.070635,7357.443386,7358.539832,7361.253986,7351.790642,7361.253158,23589.541125,23587.896243,23570.809137,23566.185747,23559.078592,23564.091116,23567.974514,23567.808841,23567.692157,23566.422486,14226.730489,14239.927737,14232.587368,14233.739231,14238.266471,14248.617194,14240.853276,14259.348142,14257.956495,14271.286100,622.585028,623.581825,623.270867,624.454417,624.816461,626.164126,626.118377,626.105657,625.686095,626.871608,1873.780534,1874.402735,1876.599001,1876.037480,1876.887803,1875.801928,1875.876145,1876.735300,1877.370379,1877.116812,1.633468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16280,2019-09-06,ECOMM,AUT,14,23570.655319,16462.000000,9982.000000,1872.259574,37,37,7358.600642,7694.254317,7361.804970,7358.069654,7351.070635,73

### Resampling data and droping columns 
in this function we resample our data on weekly basis  add date values, drop unwanted columns ,
drop columns with very low correlation and fill nan with mean 

In [104]:
for i in range(len(df_list)):
    ml_data1 = df_list[i].copy()
    name = ml_data1[split_column].unique()
    #ml_data = ml_data1.resample('W', on='DATE').sum()
    ml_data=ml_data1.copy()
    #ml_data = add_values(ml_data)
    ml_data = ml_data.drop(cols_drop, axis=1)
    ml_data.reset_index(inplace = True)
    ml_data[split_column] = name[0]

    corr_matrix = ml_data.corr()
    column_of_interest = 'logarithm_base2'
    corr = corr_matrix[column_of_interest].loc[corr_matrix[column_of_interest] != 1]
    threshold = 0.3
    columns_to_drop = corr[corr < threshold].index.tolist()
    ml_data = ml_data.drop(columns_to_drop, axis=1)
    ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())

    try:
        Dataset.Tabular.register_pandas_dataframe(dataframe = ml_data,
                                               target = az_store,
                                               name = name[0]+'_DATA_FOR_ML',
                                                description=name[0]+'READY FOR ML')
    except Exception as ex:
        print('Error register_pandas_dataframe: ', ex)


<ipython-input-104-aa531df1524f>:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
Message: rslex failed, falling back to clex.
Payload: {"pid": 119, "source": "azureml.dataprep", "version": "4.8.6", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_loggerfactory.py, line 132 in function wrapper.\n<ipython-input-104-aa531df1524f>, line 20 in function <module>.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.15.2"}


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/919f2efc-88f4-4f73-b6c7-784e34e5a04f/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Message: rslex failed, falling back to clex.
Payload: {"pid": 119, "source": "azureml.dataprep", "version": "4.8.6", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_loggerfactory.py, line 132 in function wrapper.\n<ipython-input-104-aa531df1524f>, line 20 in function <module>.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.15.2"}


Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/ba9ceb63-b841-4d56-8a65-cbb4d231af16/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Message: rslex failed, falling back to clex.
Payload: {"pid": 119, "source": "azureml.dataprep", "version": "4.8.6", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_loggerfactory.py, line 132 in function wrapper.\n<ipython-input-104-aa531df1524f>, line 20 in function <module>.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.15.2"}


...

Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/11de1c90-b2ec-4887-ba0b-65c26f4adb87/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Message: rslex failed, falling back to clex.
Payload: {"pid": 119, "source": "azureml.dataprep", "version": "4.8.6", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_loggerfactory.py, line 132 in function wrapper.\n<ipython-input-104-aa531df1524f>, line 20 in function <module>.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.15.2"}


Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/70ac803e-9323-451c-b21f-1d6662ca5a06/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


Successfully created and registered a new dataset.


### complete the experiment

In [105]:
experiment_run.complete()